### 1.Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc

import requests
from bs4 import BeautifulSoup

import pandas as pd
import time
import random
import re
from datetime import datetime

### 2.코스메 주간 랭킹

In [2]:
# 크롤링 할 단어
brand_names = []
product_names = []
category_names = []
rating_list = []
review_list = []
price_list = []
onsale_date = []
url_list = []

# base_url = "https://www.cosme.net/ranking/products/page/"

# 사이트 이동
for page_num in range(10):
    # url = base_url + str(page_num)
    url = f"https://www.cosme.net/ranking/products/page/{page_num}/week/1"

    response = requests.get(url)
    time.sleep(random.uniform(5, 10))
    soup = BeautifulSoup(response.text, "html.parser")

    # '집계기간'이 포함된 태그 찾기
    date_range_tag = soup.find(string=lambda text: "集計期間" in text)

    # 찾은 태그에서 텍스트 추출
    if date_range_tag:
        date_range = date_range_tag.strip()[5:]  
    
    # 브랜드
    brands = soup.find_all("span", class_="brand")
    for span in brands:
        a_tag = span.find("a")  # <span> 안의 <a> 태그 추출
        if a_tag:
            brand_names.append(a_tag.text)
    
    # 아이템
    items = soup.find_all("span", class_="item")
    for span in items:
        a_tag = span.find("a")  
        if a_tag:
            product_names.append(a_tag.text)

    # 카테고리
    categorys = soup.find_all("span", class_="category")
    for span in categorys:
        a_tag = span.find("a")  
        if a_tag:
            category_names.append(a_tag.text)

    # 평점
    rating_point_tags = soup.find_all("div", class_="rating-point clearfix")
    for rating_point_tag in rating_point_tags:
        rating_tag = rating_point_tag.find("p", class_=["rating", "reviewer-average", "arg-5", "arg-6"])
        if rating_tag:
            rating = rating_tag.get_text().strip()  # 평점 텍스트 추출
            rating_list.append(rating)

    # 리뷰 수
    review_tags = soup.find_all("a", class_="count")
    for review_tag in review_tags:
        review_count = review_tag.get_text().strip()  # 리뷰 수 텍스트
        review_list.append(review_count)

    # URL
    urls = soup.find_all("dd", class_="pic")

    for url in urls:
        a_tag = url.find("a")  # <a> 태그를 찾기
        if a_tag:
            url_list.append(a_tag.get("href"))  # href 속성 추출

    # 가격과 발매일 정보를 모두 추출
    price_tags = soup.find_all("p", class_="price")
    release_date_tags = soup.find_all("p", class_="onsale")

    # 가격과 발매일 정보가 각각 동일한 개수로 존재한다고 가정
    for price_tag, release_date_tag in zip(price_tags, release_date_tags):
        price = price_tag.get_text().strip()  # 가격 텍스트
        release_date = release_date_tag.get_text().strip()  # 발매일 텍스트
        price_list.append(price)
        onsale_date.append(release_date)
    
    print(f'page : {page_num} DONE')

print("-" * 50)
print(f"brand_names 길이: {len(brand_names)}")
print(f"product_names 길이: {len(product_names)}")
print(f"category_names 길이: {len(category_names)}")
print(f"rating_list 길이: {len(rating_list)}")
print(f"review_list 길이: {len(review_list)}")
print(f"price_list 길이: {len(price_list)}")
print(f"onsale_date 길이: {len(onsale_date)}")
print(f"url_list 길이: {len(url_list)}")

page : 0 DONE
page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
page : 6 DONE
page : 7 DONE
page : 8 DONE
page : 9 DONE
--------------------------------------------------
brand_names 길이: 100
product_names 길이: 100
category_names 길이: 100
rating_list 길이: 100
review_list 길이: 100
price_list 길이: 100
onsale_date 길이: 100
url_list 길이: 100


In [3]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

df = pd.DataFrame({
    "브랜드" : brand_names,
    "제품명" : product_names,
    "카테고리" : category_names,
    "평점" : rating_list,
    "리뷰수" : review_list,
    "가격(円)" : price_list,
    "발매일" : onsale_date,
    'url' : url_list,
    '업로드일' : now
})
# 순위
df = df.reset_index()
df.rename(columns = {'index' : '순위'}, inplace=True)
df['순위'] = df['순위'] + 1

# 발매일에서 날짜 부분만 추출
df['발매일'] = df['발매일'].apply(lambda x: re.sub(r'発売日：', '', x))

# URL에서 상품 ID 추출
df['상품ID'] = df['url'].apply(lambda x: re.search(r'products/(\d+)', x).group(1))

# 가격 전처리 함수
def extract_price(price_text):
    # 정규표현식을 사용하여 숫자만 추출
    price_numbers = re.findall(r'(\d+,?\d*)', price_text)
    if price_numbers:
        # 쉼표가 있는 경우 처리
        final_price = price_numbers[-1].replace(',', '')
        return final_price
    return "0"  # 숫자가 없는 경우 기본값

# 가격 컬럼 전처리
df['가격(円)'] = df['가격(円)'].apply(extract_price)
print(df.shape)
df.head()

(100, 11)


,순위,브랜드,제품명,카테고리,평점,리뷰수,가격(円),발매일,url,업로드일,상품ID
0,1,クレ・ド・ポー ボーテ,ヴォワールコレクチュールｎ,化粧下地,5.7,14343,7700,2020/3/21,https://www.cosme.net/products/10188648/,2025-02-20 23:26:48,10188648
1,2,コスメデコルテ,ルージュデコルテ クリームグロウ,口紅,5.2,914,5500,2025/1/16,https://www.cosme.net/products/10265098/,2025-02-20 23:26:48,10265098
2,3,ロージーローザ,マルチファンデパフ ２Ｐ,パフ・スポンジ,5.5,4647,638,2023/11/1,https://www.cosme.net/products/10243404/,2025-02-20 23:26:48,10243404
3,4,ロージーローザ,リアルックミラー,その他メイクグッズ,5.1,3449,495,2015/10/27,https://www.cosme.net/products/10101720/,2025-02-20 23:26:48,10101720
4,5,キャンメイク,クリーミータッチライナー,ジェルアイライナー,5.4,19806,715,2018/3/1,https://www.cosme.net/products/10147158/,2025-02-20 23:26:48,10147158


In [5]:
safe_date_range = date_range.replace("/", "-").replace("〜", "_")
df.to_csv(f"cosme_data/{safe_date_range}_cosme_ranking.csv", index=False, encoding="utf-8-sig")
df.head()

,순위,브랜드,제품명,카테고리,평점,리뷰수,가격(円),발매일,url,업로드일,상품ID
0,1,クレ・ド・ポー ボーテ,ヴォワールコレクチュールｎ,化粧下地,5.7,14343,7700,2020/3/21,https://www.cosme.net/products/10188648/,2025-02-20 23:26:48,10188648
1,2,コスメデコルテ,ルージュデコルテ クリームグロウ,口紅,5.2,914,5500,2025/1/16,https://www.cosme.net/products/10265098/,2025-02-20 23:26:48,10265098
2,3,ロージーローザ,マルチファンデパフ ２Ｐ,パフ・スポンジ,5.5,4647,638,2023/11/1,https://www.cosme.net/products/10243404/,2025-02-20 23:26:48,10243404
3,4,ロージーローザ,リアルックミラー,その他メイクグッズ,5.1,3449,495,2015/10/27,https://www.cosme.net/products/10101720/,2025-02-20 23:26:48,10101720
4,5,キャンメイク,クリーミータッチライナー,ジェルアイライナー,5.4,19806,715,2018/3/1,https://www.cosme.net/products/10147158/,2025-02-20 23:26:48,10147158


### 3. 코스메 브랜드

In [6]:
brand_list = []
brand_link_list = []

for page_num in range(1, 6):
    url = f"https://www.cosme.net/brandfanclub/brandlist/kana/000/p/{page_num}#list"

    response = requests.get(url)
    time.sleep(random.uniform(5, 10))
    soup = BeautifulSoup(response.text, "html.parser")

    # 특정 클래스 'brd' 내의 a 태그 모두 찾기
    span_tags = soup.find_all("span", class_="brd")

    links = []
    for span in span_tags:
        a_tag = span.find("a")  # 각 span 태그 내부에서 a 태그 찾기
        if a_tag:
            links.append((a_tag.text, a_tag["href"]))

    # 결과 출력
    if links:
        for text, link in links:
            brand_list.append(text)
            brand_link_list.append(link)
    else:
        print("링크를 찾을 수 없습니다.")

brand_df = pd.DataFrame({
    '브랜드명(일본)' : brand_list,
    '브랜드페이지' : brand_link_list
})
brand_df.head()

,브랜드명(일본),브랜드페이지
0,AHRES(アーレス),https://www.cosme.net/brand/brand_id/127466/to...
1,アイプチ,https://www.cosme.net/brand/brand_id/109101/to...
2,i’m meme(アイムミミ),https://www.cosme.net/brand/brand_id/124093/to...
3,AVEDA(アヴェダ),https://www.cosme.net/brand/brand_id/590/top/nt
4,アクセーヌ,https://www.cosme.net/brand/brand_id/216/top/nt


In [7]:
brand_df.to_csv("cosme_data/cosme_brand.csv", index=False, encoding="utf-8-sig")

### 제품 상세

In [8]:
product_list = df['상품ID'].to_list()

In [11]:
# 제품 정보를 저장할 리스트 생성
all_product_details = []

# 각 제품에 대해 정보 수집
for product_id in product_list:  
    url = f"https://www.cosme.net/products/{product_id}/"

    # 정보 저장할 딕셔너리 생성
    product_details = {'product_id': product_id}  # product_id는 첫 번째 열로 추가

    # 요청 보내고 응답 받기
    response = requests.get(url)
    time.sleep(random.uniform(5, 10))  # 요청 간격 두기
    soup = BeautifulSoup(response.text, "html.parser")

    # 특정 ID와 클래스명을 가진 div 태그 찾기
    maker = soup.find("dl", class_="maker clearfix")

    # maker 정보 추출
    if maker:
        maker_name = maker.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['メーカー'] = maker_name  # "メーカー" 항목에 해당하는 값 저장

    # 브랜드 정보 추출
    brand = soup.find("dl", class_="brand-name clearfix")

    if brand:
        brand_name = brand.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['ブランド名'] = brand_name

    # 아이템 카테고리 정보 추출
    category = soup.find("dl", class_="item-category clearfix")

    if category:
        # 모든 <a> 태그의 텍스트를 가져와서 " > "로 연결
        category_names = " > ".join([a.text.strip() for a in category.find_all("a")])
        product_details['アイテムカテゴリ'] = category_names

    # 베스트 코スメ 어워드 정보 추출
    bestcosme = soup.find("dl", class_="bestcosme clearfix")

    if bestcosme:
        # <ul> 내 모든 <li> 항목을 가져오기
        bestcosme_awards = {}
        for idx, li in enumerate(bestcosme.find_all("li"), start=1):
            award_text = li.find("a").text.strip()  # 링크 텍스트
            award_url = li.find("a")["href"]  # 링크 URL
            bestcosme_awards[f"BestCosme_Award_{idx}"] = award_text

        # BestCosme 항목 추가
        product_details['ベストコスメ'] = bestcosme_awards

    # 랭킹IN 정보 추출
    rankingIn = soup.find("dl", class_="ranking-in clearfix")

    if rankingIn:
        product_details['ランキングIN'] = rankingIn.find("a").text.strip().replace("\u3000", " ")

    # 제품 정보 리스트에 추가
    all_product_details.append(product_details)

In [12]:
# all_product_details 리스트를 DataFrame으로 변환
df_detail = pd.DataFrame(all_product_details)
df_detail.head()

,product_id,メーカー,ブランド名,アイテムカテゴリ,ベストコスメ,ランキングIN
0,10188648,資生堂インターナショナル,クレ・ド・ポー ボーテ,ベースメイク > 化粧下地・コンシーラー > 化粧下地,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 殿堂...,化粧下地ランキング 1位
1,10265098,コスメデコルテ,コスメデコルテ,メイクアップ > 口紅・グロス・リップライナー > 口紅,NaN,口紅ランキング 3位
2,10243404,ロージーローザ,ロージーローザ,美容グッズ・美容家電 > メイクアップグッズ > パフ・スポンジ,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 ベス...,パフ・スポンジランキング 1位
3,10101720,ロージーローザ,ロージーローザ,美容グッズ・美容家電 > メイクアップグッズ > その他メイクグッズ,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 ベス...,その他メイクグッズランキング 1位
4,10147158,井田ラボラトリーズ,キャンメイク,メイクアップ > アイライナー > ジェルアイライナー,{'BestCosme_Award_1': '@cosmeベストコスメアワード2022 殿堂...,ジェルアイライナーランキング 1位


In [14]:
df_detail.columns = ["상품ID", "제조사", "브랜드명", "카테고리", "베스트_코스메", "카테고리_랭킹"]

In [16]:
# DataFrame을 CSV로 저장
df_detail.to_csv("cosme_data/cosme_ranking_details.csv", index=False, encoding="utf-8-sig")